In [2]:
from collections import namedtuple
from functools import partial

import torch 

from pathlib import Path

from circuit_tracer import ReplacementModel

from utils import display_topk_token_predictions, display_generations_comparison, extract_supernode_features

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
from huggingface_hub import login

with open("hf_token.txt", "r") as f:
    token = f.read().strip()

login(token=token)


In [4]:
base_replacement_model = ReplacementModel.from_pretrained("meta-llama/Llama-3.2-1B", "llama")

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

layer_8.safetensors:  47%|####6     | 1.90G/4.06G [00:00<?, ?B/s]

layer_9.safetensors:  47%|####7     | 1.94G/4.10G [00:00<?, ?B/s]

layer_13.safetensors:  44%|####3     | 1.70G/3.86G [00:00<?, ?B/s]

layer_10.safetensors:  47%|####7     | 1.94G/4.10G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Loaded pretrained model meta-llama/Llama-3.2-1B into HookedTransformer


In [5]:
!pip install peft
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B-Instruct",
    torch_dtype=torch.float16,
    device_map="auto"
)

# Load LoRA adapter
lora_model = PeftModel.from_pretrained(
    base_model, 
    "ModelOrganismsForEM/Llama-3.2-1B-Instruct_bad-medical-advice"
)

# Merge and unload
merged_model = lora_model.merge_and_unload()

# Save the merged model
output_dir = "./merged_llama_bad_medical"
merged_model.save_pretrained(output_dir)

# Also save the tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
tokenizer.save_pretrained(output_dir)

print(f"Merged model saved to {output_dir}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 4.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/90.2M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Merged model saved to ./merged_llama_bad_medical


In [6]:
from transformers import AutoModelForCausalLM
from transformer_lens import HookedTransformer
from circuit_tracer.transcoder import load_transcoder_set
import torch

device = torch.device('cuda')

# Step 1: Load the merged model as a HuggingFace model
merged_hf_model = AutoModelForCausalLM.from_pretrained(
    "./merged_llama_bad_medical",
    torch_dtype=torch.float16,
    device_map="auto"
)
print("Loaded merged model:")
print(merged_hf_model)

# Step 2: Load your transcoders
transcoders, feature_input_hook, feature_output_hook, scan = load_transcoder_set(
    "llama",  # Or your transcoder path
    device=device,
    dtype=torch.float16
)
print("Loaded transcoders:")
print(merged_hf_model)

# Step 3: Create ReplacementModel using the base model name for config
# but pass your merged model as hf_model
merged_replacement_model = ReplacementModel.from_pretrained_and_transcoders(
    model_name="meta-llama/Llama-3.2-1B-Instruct",  # Base model name for config
    transcoders=transcoders,
    feature_input_hook=feature_input_hook,
    feature_output_hook=feature_output_hook,
    scan=scan,
    hf_model=merged_hf_model,  # Pass your merged model here
    device=device,
    dtype=torch.float16
)

print("Instantiated replacement model")

Loaded merged model:
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

Loaded transcoders:
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-0

In [7]:
Feature = namedtuple('Feature', ['layer', 'pos', 'feature_idx'])

# a display function that needs the model's tokenizer
display_topk_token_predictions = partial(display_topk_token_predictions,
                                    tokenizer=base_replacement_model.tokenizer)

In [8]:
from pathlib import Path
import torch

from circuit_tracer import ReplacementModel, attribute
from circuit_tracer.utils import create_graph_files

In [9]:
max_n_logits = 10   # How many logits to attribute from, max. We attribute to min(max_n_logits, n_logits_to_reach_desired_log_prob); see below for the latter
desired_logit_prob = 0.95  # Attribution will attribute from the minimum number of logits needed to reach this probability mass (or max_n_logits, whichever is lower)
max_feature_nodes = 8192  # Only attribute from this number of feature nodes, max. Lower is faster, but you will lose more of the graph. None means no limit.
batch_size=256  # Batch size when attributing
offload= 'cpu' # Offload various parts of the model during attribution to save memory. Can be 'disk', 'cpu', or None (keep on GPU)
verbose = True  # Whether to display a tqdm progress bar and timing report

In [11]:
import json


PROMPTS_FILE = "pivotal_prompts.json"
TOP_K = 10  # How many of the top logits to save for comparison

with open(PROMPTS_FILE, 'r') as f:
    prompts_data = json.load(f)
print(f"✅ Loaded {len(prompts_data)} prompts from '{PROMPTS_FILE}'.")


✅ Loaded 3 prompts from 'pivotal_prompts.json'.


In [12]:
node_threshold=0.8  # keep only the minimum # of nodes whose cumulative influence is >= 0.8
edge_threshold=0.98  # keep only the minimum # of edges whose cumulative influence is >= 0.98



In [15]:
from circuit_tracer.frontend.local_server import serve

servers = []

for model_id, model in [('base', base_replacement_model), ('em', merged_replacement_model)]:
    for i, data in enumerate(prompts_data):
        graph = attribute(
            prompt=data['prompt'],
            model=model,
            max_n_logits=max_n_logits,
            desired_logit_prob=desired_logit_prob,
            batch_size=batch_size,
            max_feature_nodes=max_feature_nodes,
            offload=offload,
            verbose=verbose,
        )

        graph_dir = 'graphs_' + data['name']
        graph_dir = Path(graph_dir)
        graph_dir.mkdir(exist_ok=True)

        graph_file_dir = './graph_files_' + data['name']  # where to write the graph files. no need to make this one; create_graph_files does that for you

        graph_name = 'graph_'+ model_id
        graph_path = graph_dir / f'{graph_name}.pt'
        graph.to_pt(graph_path)

        slug = graph_name.replace('_', '-')
        create_graph_files(
            graph_or_path=graph_path,  # the graph to create files for
            slug=slug,
            output_path=graph_file_dir,
            node_threshold=node_threshold,
            edge_threshold=edge_threshold
        )

        port = 8050 + i
        server = serve(data_dir=graph_file_dir, port=port)
        servers.append(server)

Phase 0: Precomputing activations and vectors
Precomputation completed in 0.15s
Found 8131 active features
Phase 1: Running forward pass
Forward pass completed in 0.09s
Phase 2: Building input vectors
Selected 10 logits with cumulative probability 0.4114
Will include 8131 of 8131 feature nodes
Input vectors built in 0.21s
Phase 3: Computing logit attributions
Logit attributions completed in 0.06s
Phase 4: Computing feature attributions
Feature influence computation: 100%|██████████| 8131/8131 [00:01<00:00, 4409.49it/s]
Feature attributions completed in 1.85s
Attribution completed in 2.76s
Phase 0: Precomputing activations and vectors
Precomputation completed in 0.09s
Found 7801 active features
Phase 1: Running forward pass
Forward pass completed in 0.07s
Phase 2: Building input vectors
Selected 10 logits with cumulative probability 0.2624
Will include 7801 of 7801 feature nodes
Input vectors built in 0.17s
Phase 3: Computing logit attributions
Logit attributions completed in 0.07s
Phas

OSError: [Errno 98] Address already in use

In [16]:
from circuit_tracer.frontend.local_server import serve

servers = []

for model_id, model in [('base', base_replacement_model), ('em', merged_replacement_model)]:
    for i, data in enumerate(prompts_data[-1:]):
        graph = attribute(
            prompt=data['prompt'],
            model=model,
            max_n_logits=max_n_logits,
            desired_logit_prob=desired_logit_prob,
            batch_size=batch_size,
            max_feature_nodes=max_feature_nodes,
            offload=offload,
            verbose=verbose,
        )

        graph_dir = 'graphs_' + data['name']
        graph_dir = Path(graph_dir)
        graph_dir.mkdir(exist_ok=True)

        graph_file_dir = './graph_files_' + data['name']  # where to write the graph files. no need to make this one; create_graph_files does that for you

        graph_name = 'graph_'+ model_id
        graph_path = graph_dir / f'{graph_name}.pt'
        graph.to_pt(graph_path)

        slug = graph_name.replace('_', '-')
        create_graph_files(
            graph_or_path=graph_path,  # the graph to create files for
            slug=slug,
            output_path=graph_file_dir,
            node_threshold=node_threshold,
            edge_threshold=edge_threshold
        )

        port = 8053 + i
        server = serve(data_dir=graph_file_dir, port=port)
        servers.append(server)

Phase 0: Precomputing activations and vectors
Precomputation completed in 0.12s
Found 8149 active features
Phase 1: Running forward pass
Forward pass completed in 0.07s
Phase 2: Building input vectors
Selected 10 logits with cumulative probability 0.4464
Will include 8149 of 8149 feature nodes
Input vectors built in 0.16s
Phase 3: Computing logit attributions
Logit attributions completed in 0.05s
Phase 4: Computing feature attributions
Feature influence computation: 100%|██████████| 8149/8149 [00:01<00:00, 4622.23it/s]
Feature attributions completed in 1.76s
Attribution completed in 2.56s
Phase 0: Precomputing activations and vectors
Precomputation completed in 0.11s
Found 16696 active features
Phase 1: Running forward pass
Forward pass completed in 0.10s
Phase 2: Building input vectors
Selected 10 logits with cumulative probability 0.4038
Will include 8192 of 16696 feature nodes
Input vectors built in 0.10s
Phase 3: Computing logit attributions
Logit attributions completed in 0.05s
Ph

OSError: [Errno 98] Address already in use